In [1]:
import requests
import pandas as pd
from pandas.core.frame import DataFrame
import csv
import time
import datetime

## To change csv file path here for testing( I provied 3 paths: "test_1.csv", "test_2.csv" and "test_3.csv", or you can use your own path).

In [2]:
df = pd.read_csv('data_for_TradeWater.csv') 

In [3]:
df.Export_Date = pd.to_datetime((df.Export_Date))

## Sorted this table by its Export_Date.

In [4]:
df.sort_values(by='Export_Date')

,Export_Date,Item_ID,Status
0,2017-01-31,201,Open
1,2017-01-31,202,Closed
2,2017-01-31,203,Open
3,2017-02-07,201,Open
4,2017-02-07,202,Closed
5,2017-02-07,203,Open
6,2017-02-14,201,Pending
7,2017-02-14,202,Closed
8,2017-02-14,203,Open
11,2017-02-21,203,Open


In [5]:
df_copy = df

In [6]:
print(df_copy)

   Export_Date  Item_ID   Status
0   2017-01-31      201     Open
1   2017-01-31      202   Closed
2   2017-01-31      203     Open
3   2017-02-07      201     Open
4   2017-02-07      202   Closed
5   2017-02-07      203     Open
6   2017-02-14      201  Pending
7   2017-02-14      202   Closed
8   2017-02-14      203     Open
9   2017-02-21      201   Closed
10  2017-02-21      202   Closed
11  2017-02-21      203     Open
12  2017-02-28      201   Closed
13  2017-02-28      202   Closed
14  2017-02-28      203     Open
15  2017-03-07      201  Pending
16  2017-03-07      202   Closed
17  2017-03-07      203     Open
18  2017-03-14      201   Closed
19  2017-03-14      202   Closed
20  2017-03-14      203     Open


## Sorted all the Item_ID and saved them to a list to make sure the sequences of final results.

In [7]:
Item_ID_list = sorted(list(set(df_copy['Item_ID'])))

In [8]:
print(Item_ID_list)

[201, 202, 203]


## Conbined the Item_ID and Status columns as a Key for future processes.

In [9]:
df_copy.Item_ID = df_copy.Item_ID.astype(str)
df_copy['Key'] = df_copy['Item_ID'] + " " + df_copy['Status']

In [10]:
data_set = set(zip(df_copy.Key, df_copy.Export_Date))

In [11]:
print(data_set)

{('202 Closed', Timestamp('2017-02-28 00:00:00')), ('203 Open', Timestamp('2017-01-31 00:00:00')), ('203 Open', Timestamp('2017-03-07 00:00:00')), ('202 Closed', Timestamp('2017-01-31 00:00:00')), ('201 Open', Timestamp('2017-01-31 00:00:00')), ('202 Closed', Timestamp('2017-02-21 00:00:00')), ('203 Open', Timestamp('2017-02-14 00:00:00')), ('203 Open', Timestamp('2017-02-07 00:00:00')), ('201 Closed', Timestamp('2017-03-14 00:00:00')), ('202 Closed', Timestamp('2017-02-07 00:00:00')), ('201 Open', Timestamp('2017-02-07 00:00:00')), ('201 Closed', Timestamp('2017-02-28 00:00:00')), ('201 Closed', Timestamp('2017-02-21 00:00:00')), ('203 Open', Timestamp('2017-03-14 00:00:00')), ('201 Pending', Timestamp('2017-03-07 00:00:00')), ('201 Pending', Timestamp('2017-02-14 00:00:00')), ('202 Closed', Timestamp('2017-03-14 00:00:00')), ('203 Open', Timestamp('2017-02-21 00:00:00')), ('203 Open', Timestamp('2017-02-28 00:00:00')), ('202 Closed', Timestamp('2017-02-14 00:00:00')), ('202 Closed', 

## This method is used to get the first open date for each item by getting all its open dates and picking the least as well as the earliest one( the sequence of this result is corresponding with the sorted Item_ID list).

In [12]:
def find_first_open_date(data_set, Item_ID_list, item_first_open_date):
    for id in Item_ID_list:
        s = str(id) + ' ' + 'Open'
        all_open_date_for_specific_ID_list = list()
        for k in data_set:
            if(s == k[0]):
                all_open_date_for_specific_ID_list.append(k[1])
            else:
                continue
#         print(all_open_date_for_specific_ID_list)
        if(all_open_date_for_specific_ID_list):
            item_first_open_date.append(min(all_open_date_for_specific_ID_list))
        else:
            item_first_open_date.append('N/A')

In [13]:
item_first_open_date = list()
find_first_open_date(data_set, Item_ID_list, item_first_open_date)
print(item_first_open_date)

[Timestamp('2017-01-31 00:00:00'), 'N/A', Timestamp('2017-01-31 00:00:00')]


## This method is used to get the item's corresponding close date by selecting all its close dates and picking the least but larger than its first open date one.

In [14]:
def find_first_valid_close_date(data_set, Item_ID_list, item_first_open_date, item_valid_close_date):
    id_index = 0
    for id in Item_ID_list:
        s = str(id) + ' ' + 'Closed'
        all_closed_date_for_specific_ID_list = list()
        for k in data_set:
            if(s == k[0] ):
                all_closed_date_for_specific_ID_list.append(k[1])
            else:
                continue
        if(all_closed_date_for_specific_ID_list):
            target = item_first_open_date[id_index]
            if(target != 'N/A'):
                date_time_obj = datetime.datetime.strptime(str(target), '%Y-%m-%d %H:%M:%S')
                item_valid_close_date.append(min(dt for dt in all_closed_date_for_specific_ID_list if dt > date_time_obj))
            else:
                item_valid_close_date.append('N/A')
        else:
            item_valid_close_date.append('N/A')
        id_index = id_index + 1
#         print(id_index)

In [15]:
item_valid_close_date = list()
find_first_valid_close_date(data_set, Item_ID_list, item_first_open_date, item_valid_close_date)
print(item_valid_close_date)

[Timestamp('2017-02-21 00:00:00'), 'N/A', 'N/A']


In [16]:
def get_final_result(Item_ID_list, item_first_open_date, item_valid_close_date, result_dict):
    index = 0
    while index < len(Item_ID_list):
        if(item_first_open_date[index] != 'N/A'):
            if(item_valid_close_date[index] != 'N/A'):
#                 print(type(item_valid_close_date[index]))
                result = item_valid_close_date[index] - item_first_open_date[index]
                result_dict[str(Item_ID_list[index])] = str(int(result.days/7))
                print('Item '+ str(Item_ID_list[index]) +"'s fist open duration is "+ str(int(result.days/7)) + " weeks!")
            else:
                result_dict[str(Item_ID_list[index])] = 'Never closed!'
                print('Item '+ str(Item_ID_list[index]) +' is never closed!')
        else:
            result_dict[str(Item_ID_list[index])] = 'Never opened!'
            print('Item '+ str(Item_ID_list[index]) +' is never opened!')
        index = index + 1

## Final results are here!

In [17]:
result_dict = {}
get_final_result(Item_ID_list, item_first_open_date, item_valid_close_date, result_dict)

Item 201's fist open duration is 3 weeks!
Item 202 is never opened!
Item 203 is never closed!


In [18]:
print(result_dict)

{'201': '3', '202': 'Never opened!', '203': 'Never closed!'}


## To export the result dict to a csv file.

In [19]:
now = datetime.datetime.now()

csv_file = "final_results "+str(now)+".csv"
try:
    with open(csv_file, 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['Item_ID','First Open Duration(weeks)']) 
        for key, value in result_dict.items():
           writer.writerow([key, value])
except IOError:
    print("I/O error") 